In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
967993,2021-02-04,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
967994,2021-02-05,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
967995,2021-02-06,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
967996,2021-02-07,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
26407,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
26409,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
26411,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
26413,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
26415,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
600860,2021-02-04,Arkansas,Arkansas,5001,1837,36.00,5000,Arkansas,AR,Arkansas,State,3017804
600862,2021-02-05,Arkansas,Arkansas,5001,1855,36.00,5000,Arkansas,AR,Arkansas,State,3017804
600864,2021-02-06,Arkansas,Arkansas,5001,1870,37.00,5000,Arkansas,AR,Arkansas,State,3017804
600866,2021-02-07,Arkansas,Arkansas,5001,1881,37.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
967993,2021-02-04,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
967994,2021-02-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
967995,2021-02-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
967996,2021-02-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-02-08') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
384,2021-02-08,Snohomish,Washington,29431,495.00,53061,WA,County,822083,3580.05,60.21
766,2021-02-08,Cook,Illinois,461118,9579.00,17031,IL,County,5150233,8953.34,185.99
1147,2021-02-08,Orange,California,253289,3383.00,6059,CA,County,3175692,7975.87,106.53
1527,2021-02-08,Maricopa,Arizona,488687,7979.00,4013,AZ,County,4485414,10895.03,177.89
1907,2021-02-08,Los Angeles,California,1149565,18135.00,6037,CA,County,10039107,11450.87,180.64
...,...,...,...,...,...,...,...,...,...,...,...
967645,2021-02-08,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08
967764,2021-02-08,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00
967852,2021-02-08,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00
967936,2021-02-08,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
384,2021-02-08,Snohomish,Washington,29431,495.00,53061,WA,County,822083,3580.05,60.21,60.21,3580.05
766,2021-02-08,Cook,Illinois,461118,9579.00,17031,IL,County,5150233,8953.34,185.99,185.99,8953.34
1147,2021-02-08,Orange,California,253289,3383.00,6059,CA,County,3175692,7975.87,106.53,106.53,7975.87
1527,2021-02-08,Maricopa,Arizona,488687,7979.00,4013,AZ,County,4485414,10895.03,177.89,177.89,10895.03
1907,2021-02-08,Los Angeles,California,1149565,18135.00,6037,CA,County,10039107,11450.87,180.64,180.64,11450.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...
967645,2021-02-08,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08,75.08,1651.65
967764,2021-02-08,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00,0.00,4779.41
967852,2021-02-08,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00,0.00,4238.26
967936,2021-02-08,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
683347,2021-02-08,Clarke,Alabama,3336,40.00,1025,AL,County,23622,14122.43,169.33,169.33,14122.43,1
628127,2021-02-08,Hale,Alabama,2002,56.00,1065,AL,County,14651,13664.60,382.23,382.23,13664.60,2
492081,2021-02-08,Lowndes,Alabama,1271,42.00,1085,AL,County,9726,13068.06,431.83,431.83,13068.06,3
436554,2021-02-08,Etowah,Alabama,12819,273.00,1055,AL,County,102268,12534.71,266.95,266.95,12534.71,4
366606,2021-02-08,Franklin,Alabama,3914,68.00,1059,AL,County,31362,12480.07,216.82,216.82,12480.07,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
921075,2021-02-08,Platte,Wyoming,563,11.00,56031,WY,County,8393,6707.97,131.06,131.06,6707.97,19
599599,2021-02-08,Converse,Wyoming,922,14.00,56009,WY,County,13822,6670.53,101.29,101.29,6670.53,20
827630,2021-02-08,Niobrara,Wyoming,147,2.00,56027,WY,County,2356,6239.39,84.89,84.89,6239.39,21
744384,2021-02-08,Lincoln,Wyoming,1164,11.00,56023,WY,County,19830,5869.89,55.47,55.47,5869.89,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
492081,2021-02-08,Lowndes,Alabama,1271,42.00,1085,AL,County,9726,13068.06,431.83,431.83,13068.06,3,1
435270,2021-02-08,Clay,Alabama,1362,52.00,1027,AL,County,13235,10290.90,392.90,392.90,10290.90,24,2
628127,2021-02-08,Hale,Alabama,2002,56.00,1065,AL,County,14651,13664.60,382.23,382.23,13664.60,2,3
530478,2021-02-08,Greene,Alabama,827,29.00,1063,AL,County,8111,10196.03,357.54,357.54,10196.03,26,4
491441,2021-02-08,Crenshaw,Alabama,1355,49.00,1041,AL,County,13772,9838.80,355.79,355.79,9838.80,32,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529201,2021-02-08,Hot Springs,Wyoming,357,3.00,56017,WY,County,4413,8089.73,67.98,67.98,8089.73,15,19
725976,2021-02-08,Uinta,Wyoming,1988,12.00,56041,WY,County,20226,9828.93,59.33,59.33,9828.93,6,20
744384,2021-02-08,Lincoln,Wyoming,1164,11.00,56023,WY,County,19830,5869.89,55.47,55.47,5869.89,22,21
209020,2021-02-08,Teton,Wyoming,3257,7.00,56039,WY,County,23464,13880.84,29.83,29.83,13880.84,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,60.21,3580.05,15,21
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,60.21,3580.05,15,21
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,60.21,3580.05,15,21
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,60.21,3580.05,15,21
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,60.21,3580.05,15,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965704,2021-02-04,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
965705,2021-02-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
965706,2021-02-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
965707,2021-02-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
680745,2020-04-01,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,169.33,14122.43,50,1,2.00
680746,2020-04-02,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,169.33,14122.43,50,1,0.00
680747,2020-04-03,Clarke,Alabama,3,0.00,1025,AL,County,23622,12.70,0.00,169.33,14122.43,50,1,1.00
680748,2020-04-04,Clarke,Alabama,7,0.00,1025,AL,County,23622,29.63,0.00,169.33,14122.43,50,1,4.00
680749,2020-04-05,Clarke,Alabama,9,0.00,1025,AL,County,23622,38.10,0.00,169.33,14122.43,50,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
756708,2021-02-04,Crook,Wyoming,416,10.00,56011,WY,County,7584,5485.23,131.86,131.86,5498.42,7,23,1.00
756709,2021-02-05,Crook,Wyoming,417,10.00,56011,WY,County,7584,5498.42,131.86,131.86,5498.42,7,23,1.00
756710,2021-02-06,Crook,Wyoming,417,10.00,56011,WY,County,7584,5498.42,131.86,131.86,5498.42,7,23,0.00
756711,2021-02-07,Crook,Wyoming,417,10.00,56011,WY,County,7584,5498.42,131.86,131.86,5498.42,7,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
490109,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,431.83,13068.06,1,3,1.00,0.00
490110,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,431.83,13068.06,1,3,0.00,0.00
490111,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,431.83,13068.06,1,3,0.00,0.00
490112,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,431.83,13068.06,1,3,0.00,0.00
490113,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,431.83,13068.06,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488504,2021-02-04,Albany,Wyoming,3815,11.00,56001,WY,County,38880,9812.24,28.29,28.29,9853.40,23,5,24.00,0.00
488505,2021-02-05,Albany,Wyoming,3818,11.00,56001,WY,County,38880,9819.96,28.29,28.29,9853.40,23,5,3.00,0.00
488506,2021-02-06,Albany,Wyoming,3823,11.00,56001,WY,County,38880,9832.82,28.29,28.29,9853.40,23,5,5.00,0.00
488507,2021-02-07,Albany,Wyoming,3825,11.00,56001,WY,County,38880,9837.96,28.29,28.29,9853.40,23,5,2.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-02-08') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
250517,2021-02-08,Imperial,California,26518,560.00,6025,CA,County,181215,14633.45,309.03,309.03,14633.45,1,2,4.00,1.00
1907,2021-02-08,Los Angeles,California,1149565,18135.00,6037,CA,County,10039107,11450.87,180.64,180.64,11450.87,2,5,2614.00,91.00
437790,2021-02-08,Inyo,California,1124,29.00,6027,CA,County,18039,6230.94,160.76,160.76,6230.94,3,32,11.00,0.00
56077,2021-02-08,Stanislaus,California,52996,867.00,6099,CA,County,550660,9624.09,157.45,157.45,9624.09,4,10,190.00,6.00
69474,2021-02-08,Tulare,California,45520,645.00,6107,CA,County,466195,9764.15,138.35,138.35,9764.15,5,9,171.00,0.00
26067,2021-02-08,Riverside,California,282736,3377.00,6065,CA,County,2470546,11444.27,136.69,136.69,11444.27,6,6,2566.00,57.00
396913,2021-02-08,Merced,California,27424,367.00,6047,CA,County,277680,9876.12,132.17,132.17,9876.12,7,8,290.00,5.00
530420,2021-02-08,Kings,California,21297,197.00,6031,CA,County,152940,13925.07,128.81,128.81,13925.07,8,3,73.00,3.00
25389,2021-02-08,Fresno,California,91184,1224.00,6019,CA,County,999101,9126.60,122.51,122.51,9126.60,9,14,240.00,0.00
44655,2021-02-08,San Joaquin,California,64138,923.00,6077,CA,County,762148,8415.43,121.11,121.11,8415.43,10,18,519.00,30.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
920626,2021-02-08,Lassen,California,5489,18.00,6035,CA,County,30573,17953.75,58.88,58.88,17953.75,34,1,54.00,0.00
250517,2021-02-08,Imperial,California,26518,560.00,6025,CA,County,181215,14633.45,309.03,309.03,14633.45,1,2,4.00,1.00
530420,2021-02-08,Kings,California,21297,197.00,6031,CA,County,152940,13925.07,128.81,128.81,13925.07,8,3,73.00,3.00
125061,2021-02-08,San Bernardino,California,278802,2141.00,6071,CA,County,2180085,12788.58,98.21,98.21,12788.58,13,4,853.00,16.00
1907,2021-02-08,Los Angeles,California,1149565,18135.00,6037,CA,County,10039107,11450.87,180.64,180.64,11450.87,2,5,2614.00,91.00
26067,2021-02-08,Riverside,California,282736,3377.00,6065,CA,County,2470546,11444.27,136.69,136.69,11444.27,6,6,2566.00,57.00
157113,2021-02-08,Kern,California,97522,659.00,6029,CA,County,900202,10833.35,73.21,73.21,10833.35,26,7,273.00,0.00
396913,2021-02-08,Merced,California,27424,367.00,6047,CA,County,277680,9876.12,132.17,132.17,9876.12,7,8,290.00,5.00
69474,2021-02-08,Tulare,California,45520,645.00,6107,CA,County,466195,9764.15,138.35,138.35,9764.15,5,9,171.00,0.00
56077,2021-02-08,Stanislaus,California,52996,867.00,6099,CA,County,550660,9624.09,157.45,157.45,9624.09,4,10,190.00,6.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
250517,2021-02-08,Imperial,California,26518,560.00,6025,CA,County,181215,14633.45,309.03,309.03,14633.45,1,2,4.00,1.00
1907,2021-02-08,Los Angeles,California,1149565,18135.00,6037,CA,County,10039107,11450.87,180.64,180.64,11450.87,2,5,2614.00,91.00
437790,2021-02-08,Inyo,California,1124,29.00,6027,CA,County,18039,6230.94,160.76,160.76,6230.94,3,32,11.00,0.00
56077,2021-02-08,Stanislaus,California,52996,867.00,6099,CA,County,550660,9624.09,157.45,157.45,9624.09,4,10,190.00,6.00
69474,2021-02-08,Tulare,California,45520,645.00,6107,CA,County,466195,9764.15,138.35,138.35,9764.15,5,9,171.00,0.00
26067,2021-02-08,Riverside,California,282736,3377.00,6065,CA,County,2470546,11444.27,136.69,136.69,11444.27,6,6,2566.00,57.00
396913,2021-02-08,Merced,California,27424,367.00,6047,CA,County,277680,9876.12,132.17,132.17,9876.12,7,8,290.00,5.00
530420,2021-02-08,Kings,California,21297,197.00,6031,CA,County,152940,13925.07,128.81,128.81,13925.07,8,3,73.00,3.00
25389,2021-02-08,Fresno,California,91184,1224.00,6019,CA,County,999101,9126.60,122.51,122.51,9126.60,9,14,240.00,0.00
44655,2021-02-08,San Joaquin,California,64138,923.00,6077,CA,County,762148,8415.43,121.11,121.11,8415.43,10,18,519.00,30.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,309.03,14633.45,1,2,4.00,0.00
1,03/21/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,309.03,14633.45,1,2,0.00,0.00
2,03/22/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,309.03,14633.45,1,2,0.00,0.00
3,03/23/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,309.03,14633.45,1,2,0.00,0.00
4,03/24/20,Imperial,California,9,0.00,6025,CA,County,181215,4.97,0.00,309.03,14633.45,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3272,02/04/21,Lassen,California,5435,17.00,6035,CA,County,30573,17777.12,55.60,58.88,17953.75,34,1,14.00,0.00
3273,02/05/21,Lassen,California,5435,17.00,6035,CA,County,30573,17777.12,55.60,58.88,17953.75,34,1,0.00,0.00
3274,02/06/21,Lassen,California,5435,18.00,6035,CA,County,30573,17777.12,58.88,58.88,17953.75,34,1,0.00,1.00
3275,02/07/21,Lassen,California,5435,18.00,6035,CA,County,30573,17777.12,58.88,58.88,17953.75,34,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,309.03,14633.45,1,2,4.00,0.00
1,03/21/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,309.03,14633.45,1,2,0.00,0.00
2,03/22/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,309.03,14633.45,1,2,0.00,0.00
3,03/23/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,309.03,14633.45,1,2,0.00,0.00
4,03/24/20,Imperial,California,9,0.00,6025,CA,County,181215,4.97,0.00,309.03,14633.45,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3346,02/04/21,San Joaquin,California,63258,878.00,6077,CA,County,762148,8299.96,115.20,121.11,8415.43,10,18,368.00,4.00
3347,02/05/21,San Joaquin,California,63619,893.00,6077,CA,County,762148,8347.33,117.17,121.11,8415.43,10,18,361.00,15.00
3348,02/06/21,San Joaquin,California,63619,893.00,6077,CA,County,762148,8347.33,117.17,121.11,8415.43,10,18,0.00,0.00
3349,02/07/21,San Joaquin,California,63619,893.00,6077,CA,County,762148,8347.33,117.17,121.11,8415.43,10,18,0.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)